Similar to LRG, we want to minimize the following expectation

$$\underset{\phi}{\min} \mathbb{E}_{q_\phi (x)}[f(x)]$$

We need to compute the gradient of $\mathbb{E}_{q_\phi (x)}[f(x)]$ w.r.t
$\phi$. We cannot draw samples from $q_\phi$ and take the empirical mean

$$\frac{1}{K} \sum_{i=1}^K f(x^{(i)})$$

because this expression does not contain/depend on $\phi$. The gradient of
this expression w.r.t $\phi$ is 0.

Let $p_Z(z)$ be a simple distribution, e.g. Gaussian. Suppose that there exist
 a deterministic function $g: \mathcal{Z} \rightarrow \mathcal{X}$ with
 parameter $\phi$, s.t.

$$x = g_\phi(z)$$

Replace $x$ with $g(z)$

\begin{eqnarray}
\mathbb{E}_{x \sim q_\phi(x)} \left[ f(x) \right] & = & \mathbb{E}_{z \sim p_Z(z)} \left[ f(g_\phi(z)) \right] \\
& \approx & \frac{1}{K} \sum_{i = 1}^K f(g_\phi(z^{(i)}))
\end{eqnarray}

We can differentiate the new approximation w.r.t $\phi$ because it still
appears in the approximation.

Reparameterization trick usually has much lower variance than Likelihood Ratio Grad
because:
- Unlike LRG where the derivative is the derivative of $q_\phi$ w.r.t $\phi$,
the derivative here is of $f(x)$ w.r.t $\phi$. This makes the optimization
much easier (see example below).
- There is no $\log q_\phi(x)$ term. This term is very big for high
dimensional distributions and contribute significantly to the variance of LRG.